In [2]:
import torch
import pytorch_lightning as pl

from automix.data import DSD100Dataset
from automix.system import System

In [ ]:
torch.backends.cudnn.benchmark = True
pl.seed_everything(42, workers=True)

In [ ]:
dataset_name = "DSD100"
automix_model = "dmc" # Differentiable mixing console
dataset_dir = "/import/c4dm-datasets-ex/DSD100subset"
log_dir = "./logs"

In [ ]:
# setup callbacks
callbacks = [
    LogAudioCallback(),
    pl.callbacks.LearningRateMonitor(logging_interval="step"),
    pl.callbacks.ModelCheckpoint(
        filename=f"{dataset_name}-{automix_model}"
        + "_epoch-{epoch}-step-{step}",
        monitor="val/loss_epoch",
        mode="min",
        save_last=True,
        auto_insert_metric_name=False,
    ),
]

wandb_logger = WandbLogger(save_dir=log_dir, project="automix-notebook")

# create PyTorch Lightning trainer
trainer = pl.Trainer.from_argparse_args(
    args, logger=wandb_logger, callbacks=callbacks
)

# create the System
system = System(**vars(args))

In [ ]:
train_dataset = DSD100Dataset(
    dataset_dir,
    args.train_length,
    44100,
    indices=[0, 80],
    num_examples_per_epoch=1000,
)
val_dataset = DSD100Dataset(
    dataset_dir,
    args.train_length,
    44100,
    indices=[80, 90],
    num_examples_per_epoch=100,
)
test_dataset = DSD100Dataset(
    dataset_dir,
    train_length,
    44100,
    indices=[90, 100],
    num_examples_per_epoch=100,
)